## Import libraries

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder, RobustScaler, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

In [67]:
df = pd.read_csv('../../dataset_train.csv')
df = df.drop(columns=["label"])
df_test = pd.read_csv('../../dataset_test.csv')
df.head()
df_test.head()

,id,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,...,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
0,0,tcp,FIN,0.454980,534.0,268.0,254.0,252.0,2.0,1.0,...,0.0,0.0,0.0,5.0,5.0,2.0,2.0,2.0,1.0,2.0
1,1,tcp,FIN,0.648037,8854.0,268.0,254.0,252.0,4.0,1.0,...,0.0,NaN,0.0,6.0,6.0,1.0,1.0,1.0,1.0,5.0
2,2,tcp,FIN,1.120856,3440.0,642.0,254.0,252.0,5.0,3.0,...,0.0,0.0,0.0,4.0,4.0,1.0,2.0,1.0,1.0,4.0
3,3,udp,INT,0.000001,244.0,0.0,254.0,NaN,0.0,0.0,...,0.0,0.0,0.0,10.0,4.0,2.0,4.0,2.0,1.0,4.0
4,4,tcp,FIN,0.264763,1540.0,1644.0,31.0,29.0,4.0,4.0,...,NaN,0.0,0.0,13.0,11.0,10.0,7.0,6.0,1.0,7.0


In [68]:
categorical_features = ['proto', 'state', 'service','is_sm_ips_ports','is_ftp_login','attack_cat']
noncategorical_features = [col for col in df.columns.tolist() if col not in categorical_features]

In [69]:
original_train = df.copy()
le_attack_cat = LabelEncoder()
df['attack_cat'] = le_attack_cat.fit_transform(df['attack_cat'])

train_set, val_set = train_test_split(df, test_size=0.2, random_state=42)

In [70]:
from sklearn.impute import SimpleImputer

class FeatureImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', fill_value=None):
        """
        Initialize the imputer for handling missing values.

        :param strategy: The strategy to use for imputation ('mean', 'median', 'most_frequent', 'constant').
                         Default is 'mean'.
        :param fill_value: The value to use for the 'constant' strategy. Default is None.
        """
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = SimpleImputer(strategy=self.strategy, fill_value=self.fill_value)

    def fit(self, X):
        """
        Fit the imputer to the data.

        :param X: Features data with missing values
        """
        self.imputer.fit(X)

    def transform(self, X):
        """
        Transform the data by imputing the missing values.

        :param X: Features data with missing values
        :return: Data with missing values imputed
        """
        return self.imputer.transform(X)

    def fit_transform(self, X, y=None):
        """
        Fit the imputer and transform the data.

        :param X: Features data with missing values
        :return: Data with missing values imputed
        """
        return self.imputer.fit_transform(X)

    def get_imputation_statistics(self):
        """
        Get the imputation statistics (e.g., mean or median values used for imputation).

        :return: The statistics used for imputation (depending on the strategy)
        """
        return self.imputer.statistics_

In [71]:
from sklearn.impute import SimpleImputer

class FeatureImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean', fill_value=None):
        """
        Initialize the imputer for handling missing values.

        :param strategy: The strategy to use for imputation ('mean', 'median', 'most_frequent', 'constant').
                         Default is 'mean'.
        :param fill_value: The value to use for the 'constant' strategy. Default is None.
        """
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = SimpleImputer(strategy=self.strategy, fill_value=self.fill_value)

    def fit(self, X):
        """
        Fit the imputer to the data.

        :param X: Features data with missing values
        """
        self.imputer.fit(X)

    def transform(self, X):
        """
        Transform the data by imputing the missing values.

        :param X: Features data with missing values
        :return: Data with missing values imputed
        """
        return self.imputer.transform(X)

    def fit_transform(self, X, y=None):
        """
        Fit the imputer and transform the data.

        :param X: Features data with missing values
        :return: Data with missing values imputed
        """
        return self.imputer.fit_transform(X)

    def get_imputation_statistics(self):
        """
        Get the imputation statistics (e.g., mean or median values used for imputation).

        :return: The statistics used for imputation (depending on the strategy)
        """
        return self.imputer.statistics_

In [72]:
class OutlierClipper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_percentile=0.01, upper_percentile=0.99):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def fit(self, X, y=None):
        # Ensure X is a DataFrame during fitting
        if not isinstance(X, np.ndarray):
            X = np.array(X)
        self.lower_bounds = np.percentile(X, self.lower_percentile * 100, axis=0)
        self.upper_bounds = np.percentile(X, self.upper_percentile * 100, axis=0)
        return self

    def transform(self, X):
        # Ensure X is a NumPy array during transformation
        if not isinstance(X, np.ndarray):
            X = np.array(X)
        return np.clip(X, self.lower_bounds, self.upper_bounds)

In [73]:
class DuplicateRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_df = pd.DataFrame(X)
        X_unique = X_df.drop_duplicates()
        if y is not None:
            y_unique = y[X_df.index.isin(X_unique.index)]
        return X_unique.values, y_unique

In [74]:
from sklearn.feature_selection import SelectKBest, f_classif

class FeatureSelection(BaseEstimator, TransformerMixin):
    def __init__(self, k=10, score_func=f_classif):
        """
        Initialize the feature selection process.

        :param k: Number of top features to select. Default is 10.
        :param score_func: Scoring function to evaluate the features. Default is f_classif (ANOVA F-test).
        """
        self.k = k
        self.score_func = score_func
        self.selector = SelectKBest(score_func=self.score_func, k=self.k)

    def fit(self, X, y):
        """
        Fit the selector to the data.

        :param X: Features
        :param y: Target labels
        """
        print(X.shape)
        print(y.shape)
        self.selector.fit(X, y)

    def transform(self, X):
        """
        Apply the feature selection transformation.

        :param X: Features to transform
        :return: Transformed features
        """
        return self.selector.transform(X)

    def fit_transform(self, X,y):
        """
        Fit the selector and apply the transformation.

        :param X: Features
        :param y: Target labels
        :return: Transformed features
        """
        return self.selector.fit_transform(X, y)

    def get_support(self):
        """
        Get the mask of selected features.

        :return: Mask of selected features (True/False)
        """
        return self.selector.get_support()

    def get_selected_features(self):
        """
        Get the indices of the selected features.

        :return: List of selected feature indices
        """
        return self.selector.get_support(indices=True)

In [75]:
class FeatureScaling(BaseEstimator, TransformerMixin):
    def __init__(self, method="standard"):
        self.method = method
        self.scaler = None

    def fit(self, X, y=None):
        if self.method == "standard":
            self.scaler = StandardScaler().fit(X)
        elif self.method == "minmax":
            self.scaler = MinMaxScaler().fit(X)
        return self

    def transform(self, X):
        return self.scaler.transform(X) if self.scaler else X


In [76]:
class MixedEncodingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, onehot_columns=None, label_columns=None):
        """
        Parameters:
        - onehot_columns: List of column indices for one-hot encoding.
        - label_columns: List of column indices for label encoding.
        """
        self.onehot_columns = onehot_columns or []
        self.label_columns = label_columns or []
        self.onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') if self.onehot_columns else None
        self.label_encoder = LabelEncoder() if self.label_columns else None

    def fit(self, X, y=None):
        """
        Fit the transformers to the data.

        Parameters:
        - X: Input data array (2D).
        - y: Optional target labels, not used in this transformer.
        """
        if self.onehot_columns:
            # Fit one-hot encoder for the specified columns
            self.onehot_encoder.fit(X[:, self.onehot_columns])

        if self.label_columns:
            # Fit label encoder for the specified columns
            for col in self.label_columns:
                self.label_encoder.fit(X[:, col])

        return self

    def transform(self, X):
        """
        Transform the input data using the appropriate encoding methods.

        Parameters:
        - X: Input data array (2D).
        """
        X_transformed = X.copy()

        if self.onehot_columns:
            onehot_encoded = self.onehot_encoder.transform(X[:, self.onehot_columns])
            # Replace the original columns with one-hot encoded columns
            X_transformed = np.delete(X_transformed, self.onehot_columns, axis=1)
            X_transformed = np.hstack([X_transformed, onehot_encoded])

        if self.label_columns:
            for col in self.label_columns:
                X_transformed[:, col] = self.label_encoder.transform(X[:, col])

        return X_transformed

    def fit_transform(self, X, y=None):
        """
        Fit the transformers and transform the data.

        Parameters:
        - X: Input data array (2D).
        - y: Optional target labels, not used in this transformer.
        """
        self.fit(X, y)
        return self.transform(X)


In [77]:
class SMOTEHandler(BaseEstimator, TransformerMixin):
    def __init__(self, random_state=None, sampling_strategy='auto'):
        """
        Initialize the SMOTE handler.

        :param random_state: Random state for reproducibility (default is None)
        :param sampling_strategy: Defines the sampling strategy for SMOTE (default is 'auto')
        """
        self.random_state = random_state
        self.sampling_strategy = sampling_strategy
        self.smote = SMOTE(random_state=self.random_state, sampling_strategy=self.sampling_strategy)

    def fit(self, X, y):
        """
        Fit the SMOTE model to the training data.

        :param X: Feature matrix
        :param y: Target vector
        :return: self
        """
        self.smote.fit(X, y)
        return self

    def transform(self, X, y=None):
        """
        Perform the oversampling on the training data and return the balanced dataset.

        :param X: Feature matrix
        :param y: Target vector (optional, but required if using `fit`)
        :return: Balanced feature matrix X, and target vector y
        """
        if y is None:
            raise ValueError("Target vector y must be provided for transforming the dataset")
        X_resampled, y_resampled = self.smote.fit_resample(X, y)
        return X_resampled, y_resampled

    def fit_transform(self, X, y):
        """
        Fit and transform the dataset in one step.

        :param X: Feature matrix
        :param y: Target vector
        :return: Balanced feature matrix X, and target vector y
        """
        return self.smote.fit_resample(X, y)

In [78]:
from sklearn.preprocessing import Normalizer

class DataNormalizer(BaseEstimator, TransformerMixin):
    def __init__(self, norm='l2'):
        """
        Initialize the data normalizer.

        :param norm: Norm to use for normalization, can be 'l1', 'l2', or 'max'. Default is 'l2'.
        """
        self.norm = norm
        self.normalizer = Normalizer(norm=self.norm)

    def fit(self, X, y=None):
        """
        Fit the normalizer to the data.

        :param X: Feature matrix
        :param y: Target vector (optional)
        :return: self
        """
        self.normalizer.fit(X)
        return self

    def transform(self, X):
        """
        Normalize the data.

        :param X: Feature matrix
        :return: Normalized feature matrix
        """
        return self.normalizer.transform(X)

    def fit_transform(self, X, y=None):
        """
        Fit and transform the data in one step.

        :param X: Feature matrix
        :param y: Target vector (optional)
        :return: Normalized feature matrix
        """
        return self.normalizer.fit_transform(X)

In [79]:
class DimensionalityReducer(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=None):
        """
        Initialize the PCA dimensionality reducer.

        :param n_components: Number of principal components to keep.
                              If None, keeps all components.
                              Can also be a float (explained variance ratio).
        """
        self.n_components = n_components
        self.pca = PCA(n_components=self.n_components)

    def fit(self, X, y=None):
        """
        Fit the PCA model to the data.

        :param X: Feature matrix.
        :param y: Target vector (optional).
        :return: self
        """
        self.pca.fit(X)
        return self

    def transform(self, X):
        """
        Transform the data to the lower-dimensional space.

        :param X: Feature matrix.
        :return: Transformed data in lower-dimensional space.
        """
        return self.pca.transform(X)

    def fit_transform(self, X, y=None):
        """
        Fit and transform the data in one step.

        :param X: Feature matrix.
        :param y: Target vector (optional).
        :return: Transformed data in lower-dimensional space.
        """
        return self.pca.fit_transform(X)

    def explained_variance_ratio(self):
        """
        Return the explained variance ratio of each principal component.

        :return: Array of explained variance ratios for each component.
        """
        return self.pca.explained_variance_ratio_

    def components(self):
        """
        Return the principal components (eigenvectors).

        :return: Matrix of principal components.
        """
        return self.pca.components_

In [80]:
class FeatureDiscretizer(BaseEstimator, TransformerMixin):
    def __init__(self, features, bins=10, strategy='uniform'):
        self.features = features  # List of features to discretize
        self.bins = bins  # Number of bins or discretization strategy
        self.strategy = strategy  # Discretization strategy (e.g., 'uniform', 'quantile')
        self.discretizers = {}  # Store discretizers for each feature

    def fit(self, X, y=None):
        X_df = pd.DataFrame(X, columns=self.features)  # Create DataFrame for easier handling

        for feature in self.features:
            # Create discretizer based on the chosen strategy
            if self.strategy == 'uniform':
                discretizer = np.linspace(X_df[feature].min(), X_df[feature].max(), self.bins + 1)
            elif self.strategy == 'quantile':
                discretizer = np.quantile(X_df[feature], np.linspace(0, 1, self.bins + 1))
            else:
                raise ValueError(f"Invalid strategy: {self.strategy}")

            self.discretizers[feature] = discretizer

        return self

    def transform(self, X):
        X_df = pd.DataFrame(X, columns=self.features)  # Create DataFrame for easier handling

        for feature in self.features:
            discretizer = self.discretizers[feature]
            # Apply discretization to the feature
            X_df[feature] = pd.cut(X_df[feature], bins=discretizer, labels=False, include_lowest=True, duplicates='drop')

        return X_df.values # Convert back to numpy array

In [81]:
categorical_without_target = [x for x in categorical_features if x != 'attack_cat']

# Updated numeric transformer pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', FeatureImputer(strategy='median')),
    ('outlier_clipper', OutlierClipper(lower_percentile=0.01, upper_percentile=0.99)),
    ('discretizer', FeatureDiscretizer(features=noncategorical_features, bins=10, strategy='uniform')),
    ('scaler', FeatureScaling(method='standard'))
])

# Updated categorical transformer pipeline using LabelEncoder
categorical_transformer = Pipeline(steps=[
    ('imputer', FeatureImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Define preprocessor with ColumnTransformer using your custom classes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, noncategorical_features),
        ('cat', categorical_transformer, categorical_without_target) # Using categorical_without_target
    ]
)

# Final pipeline using your custom classes
pipe = Pipeline([
    ('preprocessor', preprocessor),
    # ('smote', SMOTEHandler(random_state=42, sampling_strategy='auto')),
    # ('dimensionality_reducer', DimensionalityReducer(n_components=5)),
    # ('feature_selector', FeatureSelection(k=10)),
    # ('duplicate_remover', DuplicateRemover()),
    # ('scaler', DataNormalizer()),
])

In [82]:
from collections import Counter

class ID3DecisionTree:
    def __init__(self, max_depth=None):
        """
        Initialize the ID3 Decision Tree.

        Parameters:
        - max_depth: Maximum depth of the tree (default: None, meaning no limit).
        """
        self.max_depth = max_depth
        self.tree = None

    def entropy(self, y):
        """
        Calculate the entropy of a target variable.

        Parameters:
        - y: Target vector.
        """
        counts = np.bincount(y)
        probabilities = counts / len(y)
        return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

    def information_gain(self, X_column, y):
        """
        Calculate information gain of a split on a feature.

        Parameters:
        - X_column: Feature values.
        - y: Target vector.
        """
        parent_entropy = self.entropy(y)

        # Split the data based on unique feature values
        values, counts = np.unique(X_column, return_counts=True)
        weighted_entropy = np.sum(
            [(counts[i] / len(X_column)) * self.entropy(y[X_column == value])
             for i, value in enumerate(values)]
        )
        return parent_entropy - weighted_entropy

    def best_split(self, X, y):
        """
        Find the best feature and threshold to split on.

        Parameters:
        - X: Feature matrix.
        - y: Target vector.
        """
        best_gain = -1
        best_feature = None

        for feature in range(X.shape[1]):
            gain = self.information_gain(X[:, feature], y)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature

        return best_feature

    def build_tree(self, X, y, depth=0):
        """
        Build the decision tree recursively.

        Parameters:
        - X: Feature matrix.
        - y: Target vector.
        - depth: Current depth of the tree.
        """
        # Stop conditions
        if len(np.unique(y)) == 1:
            return y[0]  # Leaf node with class label
        if self.max_depth is not None and depth >= self.max_depth:
            return Counter(y).most_common(1)[0][0]  # Majority class
        if X.shape[1] == 0:
            return Counter(y).most_common(1)[0][0]  # Majority class

        # Find the best feature to split
        feature = self.best_split(X, y)
        if feature is None:
            return Counter(y).most_common(1)[0][0]  # Majority class

        tree = {feature: {}}
        for value in np.unique(X[:, feature]):
            sub_X = X[X[:, feature] == value]
            sub_y = y[X[:, feature] == value]
            subtree = self.build_tree(
                np.delete(sub_X, feature, axis=1), sub_y, depth + 1
            )
            tree[feature][value] = subtree

        return tree

    def fit(self, X, y):
        """
        Fit the decision tree to the training data.

        Parameters:
        - X: Feature matrix.
        - y: Target vector.
        """
        self.tree = self.build_tree(X, y)

    def predict_sample(self, tree, sample):
        """
        Predict the class label for a single sample using the decision tree.

        Parameters:
        - tree: Decision tree.
        - sample: Feature vector of the sample.
        """
        # If the current node is a leaf, return its value
        if not isinstance(tree, dict):
            return tree

        # Get the feature index for the current node
        feature = next(iter(tree))
        value = sample[feature]

        # If the value is not present in the subtree, return the majority class of the subtree
        if value not in tree[feature]:
            # Calculate the majority class of the subtree
            subtree = tree[feature]
            majority_class = Counter(
                leaf for sub in subtree.values() for leaf in self.get_all_leaves(sub)
            ).most_common(1)[0][0]
            return majority_class

        # Recurse down the tree
        subtree = tree[feature][value]
        return self.predict_sample(subtree, sample)

    def get_all_leaves(self, tree):
        """
        Helper function to collect all leaf values from a subtree.

        Parameters:
        - tree: Decision tree or subtree.

        Returns:
        - List of leaf values.
        """
        if not isinstance(tree, dict):
            return [tree]
        leaves = []
        for subtree in tree.values():
            leaves.extend(self.get_all_leaves(subtree))
        return leaves

    def predict(self, X):
        """
        Predict class labels for a dataset.
        Parameters:
        - X: Feature matrix.
        """
        return np.array([self.predict_sample(self.tree, sample) for sample in X])

    def evaluate(self, X, y_true):
        """
        Evaluate the model using accuracy, recall, and precision.

        Parameters:
        - X: Feature matrix.
        - y_true: True class labels.
        """
        y_pred = self.predict(X)

        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=1)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=1)

        print(f"Accuracy: {accuracy:.8f}")
        print(f"Precision: {precision:.8f}")
        print(f"Recall: {recall:.8f}")

# Example usage:
x_train_set = train_set.drop('attack_cat', axis=1)
y_train_set = train_set['attack_cat']
x_val_set = val_set.drop('attack_cat', axis=1)
y_val_set = val_set['attack_cat']

x_train_set_processed = pipe.fit_transform(x_train_set, y_train_set)
x_val_set_processed = pipe.transform(x_val_set)

id3 = ID3DecisionTree(max_depth=5)

# Train the model
id3.fit(x_train_set_processed, y_train_set.values)

# Evaluate the model on the validation set
id3.evaluate(x_val_set_processed, y_val_set.values)

sklearn_tree = DecisionTreeClassifier(max_depth=5, criterion='entropy', random_state=42)

# Train the sklearn DecisionTree model
sklearn_tree.fit(x_train_set_processed, y_train_set.values)

# Predict using the sklearn DecisionTree model
val_predictions_sklearn = sklearn_tree.predict(x_val_set_processed)

# Evaluate the sklearn DecisionTree model
accuracy = accuracy_score(y_val_set, val_predictions_sklearn)
precision = precision_score(y_val_set, val_predictions_sklearn, average='weighted')
recall = recall_score(y_val_set, val_predictions_sklearn, average='weighted')

print("\nSklearn DecisionTreeClassifier Evaluation:")
print(f"Accuracy: {accuracy:.8f}")
print(f"Precision: {precision:.8f}")
print(f"Recall: {recall:.8f}")


c:\Python312\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


Accuracy: 0.56634064
Precision: 0.86825840
Recall: 0.56634064

Sklearn DecisionTreeClassifier Evaluation:
Accuracy: 0.69123728
Precision: 0.67871074
Recall: 0.69123728


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## EXPORT MODEL

In [83]:
with open('../../model-id3.pkl', 'wb') as file:
    pickle.dump(id3, file)

## IMPORT MODEL

In [84]:
with open ("../../model-id3.pkl", "rb") as file:
    loaded_model = pickle.load(file)

In [ ]:
training_set = df.copy()
x_training_set = training_set.drop('attack_cat', axis=1)
y_training_set = training_set['attack_cat']

x_test_set = df_test.copy()

x_training_set_processed = pipe.fit_transform(x_training_set, y_training_set)
x_test_set_processed = pipe.transform(x_test_set)

loaded_model.fit(x_training_set_processed, y_training_set.values)




c:\Python312\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [93]:
y_test_predict = loaded_model.predict(x_test_set_processed)


In [97]:
print(y_test_predict)
reversed = le_attack_cat.inverse_transform(y_test_predict)
print(reversed)

result_df = pd.DataFrame({
    "id": range(len(reversed)),
    "attack_cat": reversed
})

result_df.head()
result_df.to_csv("predictions.csv", index=False)


[3 3 3 ... 3 3 3]
['Exploits' 'Exploits' 'Exploits' ... 'Exploits' 'Exploits' 'Exploits']
